In [11]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv
import gspread
from oauth2client.service_account import ServiceAccountCredentials

load_dotenv()

#asana creds
asana_token = os.getenv('ASANA_TOKEN')
workspace_gid = os.getenv('WORKSPACE_GID')
team_gid = os.getenv('TEAM_GID')

#google sheets creds
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

#skip list
with open('users_to_skip.json', 'r', encoding='utf-8') as config_file:
    config = json.load(config_file)
users_to_skip = config.get('users_to_skip', '')
users_to_skip

['Artem Volovikov',
 'Smartics',
 'Andrey Komendant',
 'Татьяна Милова',
 'Артюшевский Дмитрий Игоревич',
 'Антонов Никита Сергеевич',
 'Дубов Константин Владимирович',
 'Евгений Подкорытов',
 'Анна Каменкова',
 'Антон Слукин',
 'Бухгалтер Арска',
 'Полина Кузнецова']

In [12]:
#asana_users_df
def get_users(asana_token, team_gid):
    
    url = f"https://app.asana.com/api/1.0/teams/{team_gid}/users"
    headers = {'Authorization': f'Bearer {asana_token}'}
    
    payload = {
    'opt_fields': 'name',
     'opt_pretty': True
     }
    
    response = requests.get(url, headers=headers, params=payload)
    users_json = response.json()
    users_df = pd.json_normalize(users_json['data'], max_level=1)
    
    rows_to_keep = []
    for _, row in users_df.iterrows():
        if row['name'] not in users_to_skip:
            rows_to_keep.append(row)
            
    users_df = pd.DataFrame(rows_to_keep).reset_index(drop=True)
    users_df = users_df.rename(columns={'gid':'user_gid'})                      
    users_df['idx'] = (users_df.index + 1).tolist()
    
    col_order = ['idx', 'user_gid', 'name']
    users_df = users_df[col_order]
    
    return users_df

users_df = get_users(asana_token, team_gid)
users_df

,idx,user_gid,name
0,1,1169819852543243,Burmistrov Dmitrii
1,2,1200684451347406,Elena Maeva
2,3,1206620019823578,Maxim Letunovskii
3,4,1208216811351938,sammaleen
4,5,1202838673949936,Александр Виноградов
5,6,1205394271497253,Александр Смирнов
6,7,1207898444509346,Алиса Гончарова
7,8,1207866432174028,Анастасия Романова
8,9,1206236521952192,Андрей Васильев
9,10,1201758129291395,Анна Воловикова


In [13]:
#get current gs instance 

table = client.open('asana_users')
sheet = table.worksheet('users')
gs_data = sheet.get('A:D')
gs_users = pd.DataFrame(gs_data[1:], columns=gs_data[0])

gs_users

,idx,user_gid,name,user_token
0,1,1169819852543243,Burmistrov Dmitrii,None
1,2,1200684451347406,Elena Maeva,2/1200684451347406/1205697720563281:04f45ed138...
2,3,1206620019823578,Maxim Letunovskii,None
3,4,1208216811351938,sammaleen,2/1208216811351938/1208785586613550:92e6872421...
4,5,1202838673949936,Александр Виноградов,2/1202838673949936/1208803692653018:ada5c2e826...
5,6,1205394271497253,Александр Смирнов,2/1205394271497253/1208804850093387:b25d672b9e...
6,7,1207898444509346,Алиса Гончарова,2/1207898444509346/1208803460085212:63667c3734...
7,8,1207866432174028,Анастасия Романова,2/1207866432174028/1208804088059801:3b2dc008ce...
8,9,1206236521952192,Андрей Васильев,2/1206236521952192/1208803516086639:ed8f35591e...
9,10,1201758129291395,Анна Воловикова,None


In [14]:
# update users_df with tokens from user_token

def upd_users_df(users_df, gs_users):
    
    users_df_upd = users_df.copy()
    users_df_upd['user_token'] = None
    
    for idx, row in users_df_upd.iterrows():
        
        user = row['name']
        
        if user in gs_users['name'].values:
            #row['user_token'] = gs_users[gs_users['name'] == user]['user_token']
            users_df_upd.loc[idx,'user_token'] = gs_users.loc[gs_users['name'] == user, 'user_token'].values[0]
        
    return users_df_upd

users_df_upd = upd_users_df(users_df, gs_users)
users_df_upd

,idx,user_gid,name,user_token
0,1,1169819852543243,Burmistrov Dmitrii,None
1,2,1200684451347406,Elena Maeva,2/1200684451347406/1205697720563281:04f45ed138...
2,3,1206620019823578,Maxim Letunovskii,None
3,4,1208216811351938,sammaleen,2/1208216811351938/1208785586613550:92e6872421...
4,5,1202838673949936,Александр Виноградов,2/1202838673949936/1208803692653018:ada5c2e826...
5,6,1205394271497253,Александр Смирнов,2/1205394271497253/1208804850093387:b25d672b9e...
6,7,1207898444509346,Алиса Гончарова,2/1207898444509346/1208803460085212:63667c3734...
7,8,1207866432174028,Анастасия Романова,2/1207866432174028/1208804088059801:3b2dc008ce...
8,9,1206236521952192,Андрей Васильев,2/1206236521952192/1208803516086639:ed8f35591e...
9,10,1201758129291395,Анна Воловикова,None


In [15]:
# update gs table with new users_df_upd

table = client.open('asana_users')
sheet = table.worksheet('users')
sheet.batch_clear(['A:D'])

data = [users_df_upd.columns.values.tolist()] + users_df_upd.values.tolist()
sheet.update(values=data, range_name='A1')

{'spreadsheetId': '1w9pbRfUU2pPqiB8oAIUs5wqPxHMjxzMcJQ6aTL0WMtM',
 'updatedRange': 'users!A1:D23',
 'updatedRows': 23,
 'updatedColumns': 4,
 'updatedCells': 81}